# General Fitbit Data exploration

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn

/Users/sf321092/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#load fitbit data
fit = pd.read_table('meas_fitbit_tracker.txt', sep='|', parse_dates=[1])
fit['date']=pd.to_datetime(fit['date'], format='%m/%d/%Y') #convert fitbit date to datetime


In [3]:
fit.shape

(1503058, 8)

In [4]:
fit.columns

Index([u'user_id', u'date', u'steps', u'distance', u'calories', u'floors',
       u'elevation', u'device_type'],
      dtype='object')

In [5]:
fit = fit.drop(['distance', 'calories', 'floors', 'elevation'], axis=1)

In [6]:
fit.shape

(1503058, 4)

In [7]:
fit.head()

,user_id,date,steps,device_type
0,12,2013-10-25,0.0,NaN
1,12,2013-08-07,0.0,NaN
2,12,2015-01-17,0.0,NaN
3,12,2014-11-20,4912.0,NaN
4,12,2014-06-30,7029.0,NaN


In [58]:
fit_mini = fit[fit.user_id<=13]

In [16]:
fit_mini.shape

(1574, 4)

In [17]:
fit_mini.user_id.unique()

array([12, 13])

In [38]:
fit_mini.user_id[12]

12

In [ ]:
fit_mini['date'].groupby(fit_mini['user_id']).transform('count')

In [59]:
#create new column of days used in fit_mini df
fit_mini['days_used']=fit_mini['date'].groupby(fit_mini['user_id']).transform('count')

/Users/sf321092/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [60]:
fit_mini.head()

,user_id,date,steps,device_type,days_used
0,12,2013-10-25,0.0,NaN,856
1,12,2013-08-07,0.0,NaN,856
2,12,2015-01-17,0.0,NaN,856
3,12,2014-11-20,4912.0,NaN,856
4,12,2014-06-30,7029.0,NaN,856


In [ ]:
#doesn't work
#trying to make an additional "days used" column
def f(row):
    if fit_mini['user_id'].ix[row,:]==user:
        return result

for user in fit_mini.user_id.unique():
    result = fit_mini.groupby('user_id').count()['date'].ix[user]
    
    fit_mini['days_used'] = fit_mini.apply(f()
    
#   fit_mini['days_used']=np.where(fit_mini['user_id']==user, result, fit_mini['days_used'])
    
#     fit_mini['days_used']= fit_mini.apply(f(result, user), axis=1)
    
#     for row in xrange(len(fit_mini.user_id)):
#         if fit_mini.user_id.loc[row]==user:
#             fit_mini['days_used']=result

#     fit_mini[fit_mini.user_id==user]['days_used'] = result
    
#     for row in xrange(len(fit_mini.user_id==user)):
#         fit_mini['days_used'] = result
#     for user in fit.user_id.unique():
#     result = fit.groupby('user_id').count()['date'].ix[user]
#     for row in xrange(len(fit.user_id==user)):
#         fit['days_used'][row] = result
    

In [49]:
fit_mini.head()

,user_id,date,steps,device_type,days_used
0,12,2013-10-25,0.0,NaN,856
1,12,2013-08-07,0.0,NaN,856
2,12,2015-01-17,0.0,NaN,856
3,12,2014-11-20,4912.0,NaN,856
4,12,2014-06-30,7029.0,NaN,856


In [50]:
fit.groupby('user_id').count().ix[12]

date           856
steps          856
device_type    174
Name: 12, dtype: int64

In [61]:
#now create a days used column for the entire fit df
fit['days_used']=fit['date'].groupby(fit['user_id']).transform('count')

In [ ]:
##here i will pause to try to run a recurrent neural network on daily data.
#so what i need to do is to create an X df with the last 60d of people's step data
#ranging from 0 to x (convert NaN to 0)
#and a Y df with blockages in coroary vs. not 
# so start by creating datasets according to these parameters 

In [ ]:
# 25% of people have at least 8 months of data, so I'm curious to determine how many have >6mo or 180 days. 
np.sum(fit.groupby('user_id').count()['date']<180)/2250.

In [ ]:
# so 13% have <180. So I'm comfortable using that as a cut-off. 
# So I will drop those with <180 days of data

In [ ]:
#testing drop based on days used on fit_mini
fit_mini.drop(fit_mini[fit_mini.days_used <750].index)

In [77]:
# First data-clean: drop users with days_used < 180
fit_clean = fit
len(fit_clean.user_id.unique())

2250

In [79]:
fit_clean.drop(fit_clean[fit_clean.days_used<180].index, inplace=True)
len(fit_clean.user_id.unique())
#So it appears we drop 2250-1938 user_ids based on days_used <180

1938

In [83]:
fit_clean.shape

(1474444, 5)

In [85]:
fit_clean.groupby('user_id').count().describe()

,date,steps,device_type,days_used
count,1938.000000,1938.000000,1938.000000,1938.000000
mean,760.807018,617.691434,759.699690,760.807018
std,444.505486,364.250526,445.023266,444.505486
min,180.000000,0.000000,4.000000,180.000000
25%,400.500000,302.000000,399.250000,400.500000
50%,698.000000,565.000000,695.000000,698.000000
75%,1004.750000,889.500000,1003.500000,1004.750000
max,2496.000000,1670.000000,2496.000000,2496.000000


In [93]:
#Now to create a dataset with only the last 60d of data for people 
#start by calculating the max date for everyone
fit_mini['max_date']=fit_mini['date'].groupby(fit_mini['user_id']).transform('max')

/Users/sf321092/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [ ]:
#testing code to drop dates < 60d from max per person
from datetime import timedelta
#fit_mini shape (1574, 6)
fit_mini.drop(fit_mini[fit_mini.date<(fit_mini.max_date-timedelta(days=59))].index)

In [111]:
fit_mini.head()

,user_id,date,steps,device_type,days_used,max_date
0,12,2013-10-25,0.0,NaN,856,2015-08-01
1,12,2013-08-07,0.0,NaN,856,2015-08-01
2,12,2015-01-17,0.0,NaN,856,2015-08-01
3,12,2014-11-20,4912.0,NaN,856,2015-08-01
4,12,2014-06-30,7029.0,NaN,856,2015-08-01


In [116]:
fit_mini['date'].groupby(fit_mini.user_id).describe()

user_id        
12       count                     856
         unique                    856
         top       2015-07-19 00:00:00
         freq                        1
         first     2013-03-29 00:00:00
         last      2015-08-01 00:00:00
13       count                     718
         unique                    718
         top       2014-05-28 00:00:00
         freq                        1
         first     2013-03-28 00:00:00
         last      2015-03-15 00:00:00
dtype: object

In [117]:
fit_mini.drop(fit_mini[fit_mini.date<(fit_mini.max_date-timedelta(days=59))].index, inplace=True)

/Users/sf321092/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [118]:
fit_mini['date'].groupby(fit_mini.user_id).describe()

user_id        
12       count                      60
         unique                     60
         top       2015-07-19 00:00:00
         freq                        1
         first     2015-06-03 00:00:00
         last      2015-08-01 00:00:00
13       count                      60
         unique                     60
         top       2015-02-07 00:00:00
         freq                        1
         first     2015-01-15 00:00:00
         last      2015-03-15 00:00:00
dtype: object

In [121]:
fit_clean['max_date']=fit_clean['date'].groupby(fit_clean['user_id']).transform('max')
fit_clean.drop(fit_clean[fit_clean.date<(fit_clean.max_date-timedelta(days=59))].index, inplace=True)
fit_clean.head()

,user_id,date,steps,device_type,days_used,max_date
685,12,2015-07-30,2317.0,MobileTrack,856,2015-08-01
686,12,2015-07-29,0.0,MobileTrack,856,2015-08-01
687,12,2015-07-21,0.0,MobileTrack,856,2015-08-01
688,12,2015-07-23,0.0,MobileTrack,856,2015-08-01
689,12,2015-07-25,0.0,MobileTrack,856,2015-08-01


In [122]:
fit_clean.shape

(116281, 6)

In [126]:
fit_clean.steps.isnull().sum()

3602

In [129]:
fit_clean.steps.fillna(0, inplace=True)

In [130]:
fit_clean.steps.isnull().sum()

0

In [131]:
# so next step is to merge fit_clean with medical conditions df and then select those with diseases/nondiseases to comprise the X, and have hte labels Y


#load med condition-full-cohort data    
med_cond = pd.read_table('surv_medical_conditions.txt', sep='|')

/Users/sf321092/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
med_cond.head()

,user_id,evisit_substudy,activity_assignment_id,started_at,completed_at,redcap_transfer,hbp,high_chol,diabetes,blockages_in_your_coronary,...,congenital_kind1,congenital_kind2,congenital_kind3,congenital_kind4,congenital_kind5,congenital_kind6,congenital_kind7,congenital_kind8,congenital_kind9,congenital_kind10
0,12,1.0,327,11MAR13:21:41:42,11MAR13:21:41:42,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14,1.0,383,09APR13:07:16:26,09APR13:07:16:26,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,4.0,4225052,17FEB16:20:31:45,17FEB16:20:32:31,0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,1.0,439,10MAR13:08:01:49,10MAR13:08:01:49,1,2.0,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17,1.0,469,19MAR13:17:10:53,19MAR13:17:10:53,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
med_cond.shape

(38840, 52)

In [135]:
med_cond.columns

Index([u'user_id', u'evisit_substudy', u'activity_assignment_id',
       u'started_at', u'completed_at', u'redcap_transfer', u'hbp',
       u'high_chol', u'diabetes', u'blockages_in_your_coronary',
       u'heart_attack', u'pvd', u'clots', u'chf', u'stroke', u'enlarged',
       u'afib', u'arrhythmia', u'murmur', u'valve', u'dental', u'congenital',
       u'congenital_kind', u'congenital_other', u'congenital_surg',
       u'pulm_hyper', u'aorta', u'other_aorta', u'marfan', u'ehlers',
       u'sleep_apnea', u'copd', u'asthma', u'cardiac', u'implant',
       u'implant_pace', u'implant_icd', u'implant_other', u'afib1', u'afib2',
       u'arrhythmia1', u'arrhythmia2', u'congenital_kind1',
       u'congenital_kind2', u'congenital_kind3', u'congenital_kind4',
       u'congenital_kind5', u'congenital_kind6', u'congenital_kind7',
       u'congenital_kind8', u'congenital_kind9', u'congenital_kind10'],
      dtype='object')

In [136]:
mini_med_cond = med_cond[['user_id', 'hbp', 'high_chol', 'diabetes', 'blockages_in_your_coronary', 'heart_attack', 'pvd', 'clots', 'chf', 'stroke', 'enlarged', 'afib', 'arrhythmia', 'murmur', 'valve', 'congenital' ,'pulm_hyper', 'aorta', 'other_aorta', 'sleep_apnea', 'copd', 'asthma', 'arrhythmia1', 'arrhythmia2']]

In [139]:
mini_med_cond.head()

,user_id,hbp,high_chol,diabetes,blockages_in_your_coronary,heart_attack,pvd,clots,chf,stroke,...,valve,congenital,pulm_hyper,aorta,other_aorta,sleep_apnea,copd,asthma,arrhythmia1,arrhythmia2
0,12,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,2.0,2.0,1.0,NaN,2.0,2.0,1.0,2.0,NaN
1,14,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN
2,15,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN
3,16,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN
4,17,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN


In [140]:
pd.crosstab(mini_med_cond.blockages_in_your_coronary, mini_med_cond.heart_attack)

heart_attack,1.0,2.0,3.0
blockages_in_your_coronary,,,
1.0,1666,1887,58
2.0,372,34158,63
3.0,27,431,94


In [155]:
# now try to create a column that combines "heart attack" or "blockages in your coronary"
def f(row):
    if row['heart_attack']==1 or row['blockages_in_your_coronary']==1:
        return int(1)
    if row['heart_attack']==2:
        return int(2)
    if row['heart_attack']==3:
        return int(3)
    if row['blockages_in_your_coronary']==2:
        return int(2)
    if row['blockages_in_your_coronary']==3:
        return int(3)

mini_med_cond['CAD']=mini_med_cond.apply(lambda row: f(row), axis=1)

/Users/sf321092/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [156]:
mini_med_cond.head()

,user_id,hbp,high_chol,diabetes,blockages_in_your_coronary,heart_attack,pvd,clots,chf,stroke,...,congenital,pulm_hyper,aorta,other_aorta,sleep_apnea,copd,asthma,arrhythmia1,arrhythmia2,CAD
0,12,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,1.0,NaN,2.0,2.0,1.0,2.0,NaN,2.0
1,14,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0
2,15,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0
3,16,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0
4,17,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0


In [157]:
pd.crosstab(mini_med_cond.CAD, mini_med_cond.heart_attack)

heart_attack,1.0,2.0,3.0
CAD,,,
1.0,2065,1887,58
2.0,0,34594,0
3.0,0,0,157


In [158]:
pd.crosstab(mini_med_cond.CAD, mini_med_cond.blockages_in_your_coronary)

blockages_in_your_coronary,1.0,2.0,3.0
CAD,,,
1.0,3613,372,27
2.0,0,34165,431
3.0,0,63,94


In [159]:
pd.crosstab(mini_med_cond.blockages_in_your_coronary, mini_med_cond.heart_attack)

heart_attack,1.0,2.0,3.0
blockages_in_your_coronary,,,
1.0,1666,1887,58
2.0,372,34158,63
3.0,27,431,94


In [167]:
mini_med_cond.CAD.value_counts()

2.0    34601
1.0     4012
3.0      157
Name: CAD, dtype: int64

In [487]:
#now merge fit and mini_med_cond df's 

union_med = pd.merge(fit_clean, mini_med_cond, how='left', on='user_id', copy=True)

In [171]:
len(union_med.user_id.unique())
#1938 people with both fitbit and med_condition surveys 

1938

In [175]:
#There are 166 people with CAD and fitbits, 1644 withough CAD and with fitbits
union_med.groupby('user_id').mean().CAD.value_counts()

2.0    1644
1.0     166
3.0       8
Name: CAD, dtype: int64

In [488]:
X_cad = union_med[union_med.CAD==1]

In [489]:
#create control group, then randomly select 166 indices from it
X_cad_control = union_med[union_med.CAD!=1]
control_user_ids = X_cad_control.user_id.unique()

In [191]:
len(control_user_ids)

1772

In [193]:
import random
X_cad_control_user_list = random.sample(control_user_ids, 166)

In [197]:
X_cad_control = X_cad_control[X_cad_control['user_id'].isin(X_cad_control_user_list)]

In [198]:
len(X_cad_control.user_id.unique())

166

In [477]:
#Great! So now my two Xvariables are X_cad and X_cad_control, which now just need to be merged, then I will pop off 
# the CAD column to be the Y varaible.
X_cad_full = pd.concat([X_cad, X_cad_control]) 


In [200]:
len(X_cad_full.user_id.unique())

332

In [204]:
X_cad_full.tail()

,user_id,date,steps,device_type,days_used,max_date,hbp,high_chol,diabetes,blockages_in_your_coronary,...,congenital,pulm_hyper,aorta,other_aorta,sleep_apnea,copd,asthma,arrhythmia1,arrhythmia2,CAD
116516,74479,2016-07-09,6794.0,Charge HR,799,2016-08-01,2.0,1.0,2.0,2.0,...,2.0,2.0,1.0,NaN,3.0,2.0,2.0,2.0,NaN,2.0
116517,74479,2016-07-13,8171.0,Charge HR,799,2016-08-01,2.0,1.0,2.0,2.0,...,2.0,2.0,1.0,NaN,3.0,2.0,2.0,2.0,NaN,2.0
116518,74479,2016-07-17,9472.0,Charge HR,799,2016-08-01,2.0,1.0,2.0,2.0,...,2.0,2.0,1.0,NaN,3.0,2.0,2.0,2.0,NaN,2.0
116519,74479,2016-07-25,5471.0,Charge HR,799,2016-08-01,2.0,1.0,2.0,2.0,...,2.0,2.0,1.0,NaN,3.0,2.0,2.0,2.0,NaN,2.0
116520,74479,2016-07-29,4772.0,Charge HR,799,2016-08-01,2.0,1.0,2.0,2.0,...,2.0,2.0,1.0,NaN,3.0,2.0,2.0,2.0,NaN,2.0


In [ ]:
#create response variable Y
Y_labels = X_cad_full.groupby('user_id').mean()['CAD']

In [394]:
Y_labels.value_counts()
#So it looks like there are 13 NaNs and 2 "3" values in the Y_labels, so I will turn both of these to "2"

1.0    166
2.0    151
3.0      2
Name: CAD, dtype: int64

In [397]:
Y_labels.replace(to_replace=3, value=2, inplace=True)

In [398]:
Y_labels.isnull().sum()

13

In [402]:
Y_labels.fillna(value=2, inplace=True)

In [405]:
Y_labels.replace(to_replace=2, value=0, inplace=True)

In [407]:
Y = Y_labels.values
#THIS IS THE RESPONSE VARAIBLE TO USE IN THE RNN

In [409]:
# now need to restructure data to be one row per user, and 60 columns wide 

In [426]:
#creating a df with repeating 0-59 to denote each person in df.pivot
sixty=pd.DataFrame([np.tile(np.arange(0,60), 332)]).T

In [428]:
sixty.columns=['ind']

In [478]:
#resetting index of X_cad_full and removing unneeded variable
X_cad_full.reset_index(inplace=True)
X_cad_full = X_cad_full.drop(['index'], axis=1)
X_cad_full.shape

(19920, 30)

In [480]:
X_cad_model = X_cad_full.join(sixty)

In [481]:
X_cad_model2 = X_cad_model[['user_id', 'steps', 'ind']]

In [484]:
#pivoting df to create 1 row per user and 60 features wide (1 feature per daily step count)
X_cad_model_arr = X_cad_model2.pivot(index='user_id', values='steps', columns='ind')

In [443]:
#resetting index and removing user_id variable
X_cad_model_arr.reset_index(inplace=True)
X_cad_model_arr.drop('user_id', axis=1, inplace=True)

In [445]:
#THIS IS THE X TO USE IN THE RNN
X_cad_model_arr.shape

(332, 60)

In [209]:
##NOW time to start trying to build the neural net!


In [469]:
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [211]:
from sklearn.cross_validation import train_test_split

In [448]:
X_train, X_test, Y_train, Y_test = train_test_split(X_cad_model_arr, Y, test_size=0.2)

In [449]:
#now, last thing, we have to format the X_train/test vars as keras wants them for RNN
x_train = X_train.values[:,:,None]
x_test = X_test.values[:,:,None]
print x_train.shape, x_test.shape

(265, 60, 1) (67, 60, 1)


In [384]:
Y_train.shape

(265,)

In [329]:
#try to build recurrent NN, LSTM

hidden_neurons = 64

model =  Sequential()
model.add(LSTM(hidden_neurons, input_shape=(60,1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [450]:
r_input = Input(shape=(60,1))
lstm = LSTM(64)(r_input)
pred = Dense(1, activation='sigmoid')(lstm)
model = Model(input=r_input, output=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [383]:
#model code for LSTM, RNN using the Functional API, with multiple layers 
r_input = Input(shape=(60,1))
lstm = LSTM(64, return_sequences=True)(r_input) #return_sequences=True outputs returns an output from this layer as shape (None, 60, 64) which allows for input to the next layer  
#another layer, need return sequences
lstm= LSTM(64)(lstm) #second layer (would add option return_sequences if intending to add another layer)
pred = Dense(1, activation='sigmoid')(lstm)
two_model = Model(input=r_input, output=pred)
two_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [451]:
model.summary()
#note that the output shape of the Input Layer matches the shape of x_train after reformatting in cell above 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 60, 1)         0                                            
____________________________________________________________________________________________________
lstm_25 (LSTM)                   (None, 64)            16896       input_11[0][0]                   
____________________________________________________________________________________________________
dense_11 (Dense)                 (None, 1)             65          lstm_25[0][0]                    
Total params: 16961
____________________________________________________________________________________________________


In [379]:
two_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 60, 1)         0                                            
____________________________________________________________________________________________________
lstm_17 (LSTM)                   (None, 60, 64)        16896       input_7[0][0]                    
____________________________________________________________________________________________________
lstm_18 (LSTM)                   (None, 64)            33024       lstm_17[0][0]                    
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 1)             65          lstm_18[0][0]                    
Total params: 49985
_______________________________________________________________________

In [458]:
model.fit(x_train, Y_train, nb_epoch=3, batch_size=10)

Epoch 1/3
265/265 [==============================] - 0s - loss: 0.7247 - acc: 0.4830     
Epoch 2/3
265/265 [==============================] - 0s - loss: 0.6952 - acc: 0.5057     
Epoch 3/3
265/265 [==============================] - 0s - loss: 0.6888 - acc: 0.4981     


In [460]:
two_model.fit(x_train, Y_train, nb_epoch=3, batch_size=10)

Epoch 1/3
265/265 [==============================] - 1s - loss: 0.7063 - acc: 0.5132     
Epoch 2/3
265/265 [==============================] - 1s - loss: 0.6915 - acc: 0.4755     
Epoch 3/3
265/265 [==============================] - 1s - loss: 0.6847 - acc: 0.5208     


In [461]:
Y_train

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,
        0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,
        0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,
        1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,
        0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,
        1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0

In [ ]:
# This is the basic pipeline and set up for the RNN, Can now experiment with adding layers, adding nodes, features, engineered features dropout etc
# Will now work to do this through a script for scalability and efficiency 


In [470]:
#playing with various parameters to see if I can increase training accuracy 
r_input = Input(shape=(60,1))
lstm = LSTM(256, return_sequences=True)(r_input) #return_sequences=True outputs returns an output from this layer as shape (None, 60, 64) which allows for input to the next layer  
#another layer, need return sequences
lstm= LSTM(256, return_sequences=True)(lstm) #second layer (would add option return_sequences if intending to add another layer)
lstm=Dropout(0.2)(lstm)
lstm= LSTM(256)(lstm)
pred = Dense(1, activation='sigmoid')(lstm)
two_model = Model(input=r_input, output=pred)
two_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [471]:
two_model.fit(x_train, Y_train, nb_epoch=3, batch_size=10)
#so model with 3 x 256 node layers and 1 dropout layer does not seem to improve accuracy

Epoch 1/3
265/265 [==============================] - 9s - loss: 0.7623 - acc: 0.4415     
Epoch 2/3
265/265 [==============================] - 9s - loss: 0.7303 - acc: 0.5321     
Epoch 3/3
265/265 [==============================] - 9s - loss: 0.7077 - acc: 0.5019     


In [ ]:
#just out of curiosity, wondering how another classifier might perform

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.cross_validation import cross_val_score

logistic = LogisticRegression()
logistic.fit(x_train, Y_train)
print "Logistic Regression score: ", cross_val_score(logistic, x_train, y_train)

rf = RandomForestClassifier()
rf.fit(x_train, y_train, n_estimators=50)
print "RF score: ", cross_val_score(rf, x_train, Y_train) 


In [492]:
pwd

u'/Data Files/2014-/Research/HeH/HeH Partners/Fitbit Analysis/Fitbit Data Analysis/160801 Galvanize Data'